# Imports

In [ ]:
%load_ext autoreload

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import hpc05

import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)
def assert_cluster_checked():
    assert run_cluster.value is True, "Command not run. Check the box above to run."

In [ ]:
import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

### Define constants

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Cluster setup

In [ ]:
run_cluster

In [ ]:
assert_cluster_checked()
hpc05.kill_remote_ipcluster()
run_cluster.value = False

In [ ]:
assert_cluster_checked()
client, dview, lview = hpc05.start_remote_and_connect(200, folder='~/two_dim_majoranas', timeout=180)
run_cluster.value = False

# Define and plot system

In [ ]:
syst_pars = {'L_down' :  500,
               'L_up' : 500,
               'Lm' : 500,
               'Lx' : 25,
               'a' :  25,
            'mu_from_bottom_of_spin_orbit_bands': True}

_=plotting_results.plot_syst(syst_pars, sns_system.dummy_params)

# Define standard parameters

In [ ]:
params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 1.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = .18,
                 Delta_right = .18,
                 B = 0.5,
                 phase = np.pi/2,
                 T = 0.025)

params = dict(**constants,
              **params_raw)

### Define keys to be varied

In [ ]:
keys_with_bounds = {"phase":[0, 2*np.pi],
                    "B":[0,1.1]}

### Define metrics to be recorded

In [ ]:
metric_params_dict = {
                       "pfaffian":{}}

### Define data folder

In [ ]:
data_folder = 'test_folder'

# Make ASS

In [ ]:
%autoreload

In [ ]:
import distributed_sns

In [ ]:
ass = ASS(keys_with_bounds,
              syst_pars, params,
              metric_params_dict)
ass=distributed_sns.AggregatesSimulationSet.load_from_file(
    '<KB_c0a2a5_><PRMS_84abf4_><PARS_3829ac_><METR_992ec1_><DIM_696722_>',
   data_folder, 100)

### Add additional dimensions

In [ ]:
alphas = [0.1, 1, 10]

In [ ]:
def _params_alpha(syst_pars, params, alpha):
    for key in ['alpha_middle', 'alpha_left', 'alpha_right']:
        if key in params:
            params[key] = alpha
        if key in syst_pars:
            syst_pars[key] = alpha
    return alpha

params_alpha = [ft.partial(_params_alpha,
                           alpha=_alpha
                           ) for _alpha in alphas]

In [ ]:
ass.add_dimension("alpha", params_alpha)

### Make learner

In [ ]:
ass.make_balancing_learner(100)
ass.load(data_folder, 100)

### Make runner with saver

In [ ]:
runner = adaptive.Runner(ass.get_balancing_learner())
ass.start_periodic_saver(runner, data_folder, interval=4)
runner.live_info()

# Plot

In [ ]:
[(l.npoints, l.loss()) for l in ass.learner.learners]

In [ ]:
%%opts Image {+framewise} [colorbar=True width=400 height=400]
%%output filename=f"./{data_folder}/spin_orbit_bottom_of_band" fig='png'

kdims, plot_dict = ass.get_plot_dict(400)
pdict = dict((k,v[:,:]) for k,v in plot_dict.items())
hv.HoloMap(plot_dict, kdims=kdims)